In [1]:
#@title setup and install libraries
!pip install -q duckduckgo-search wandb
!pip install langchain_community langchain_experimental
!pip install datasets
!pip install transformers[torch]

# Check if GPU is available and install llama-cpp-python accordingly. either on GPU or CPU
import torch
if torch.cuda.is_available():
    !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose
else:
    !pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.5/202.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
#@title mount gDrive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title initialize LlamaCpp with local mistral-7b model: Model's .gguf should be present locally
from langchain_community.llms import LlamaCpp
llm_local = LlamaCpp(
    model_path="/content/drive/MyDrive/Colab Notebooks/mistral-7b-instruct-v0.2.Q6_K.gguf",
    temperature=0.2,
    max_tokens=800,
    n_gpu_layers=64,
    n_ctx=2048
)

In [ ]:
llm_local.invoke("talk about prompt engineering")

In [ ]:
#@title imports  for finetuning the model
import os

import pandas as pd
import requests
import wandb

from IPython.display import Image
from langchain_experimental.agents import create_pandas_dataframe_agent

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks import get_openai_callback, wandb_tracing_enabled
from langchain.chat_models import ChatOpenAI as LLM
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.tools import DuckDuckGoSearchRun, tool

# only for typing
from typing import Any, Optional

from langchain.agents.agent import AgentExecutor
from langchain.callbacks.openai_info import OpenAICallbackHandler




In [ ]:
!pip install -U accelerate

In [ ]:

from datasets import load_dataset
from transformers import Trainer, TrainingArguments, LlamaTokenizer, LlamaForCausalLM

# Step 1: Load the dataset
dataset = load_dataset('imdb')

# Step 2: Preprocess the dataset
def preprocess_function(examples):
    return {
        'text': examples['text']
    }

train_dataset = dataset['train'].map(preprocess_function, batched=True)


In [ ]:

# Step 3: Tokenize the dataset
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
def tokenize_function(examples):
    tokenizer1 = tokenizer(examples['text'], padding='max_length', truncation=True)



    return tokenizer1

tokenized_datasets = train_dataset.map(tokenize_function, batched=True)


In [ ]:
tokenized_datasets

In [ ]:
!pip install accelerate -U --force-reinstall --upgrade --no-cache-dir --verbose

In [ ]:
from transformers import TrainingArguments, Trainer


# Step 4: Define training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Step 5: Load the model

# Step 6: Define a Trainer
trainer = Trainer(
    model=llm_local,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)



In [ ]:
# Step 7: Fine-tune the model
trainer.train()

# Step 8: Save the fine-tuned model
model.save_pretrained('output_dir')
tokenizer.save_pretrained('output_dir')

print("Fine-tuning completed and model saved.")
